# Test script for SomaNews Clustering
By Datetime : 2016-08-29 ~ 2016-09-05

In [1]:
import numpy as np
import pandas as pd

In [ ]:
np.c_

## Load Data
Load Data from database

In [31]:
train = pd.read_pickle("../datastore/weekly.p")
train = train.drop(['newsNum'], axis=1)
train.head()

,category,content,provider,publishedAt,title
publishedAt,,,,,
2016-09-04 23:28:00,경제,모든 사물이 인터넷으로 연결된다는 사물인터넷(IoT·Internet of Thing...,chosun,2016-09-04 23:28:00,電氣 저장했다 꺼내 쓰는 ESS… 4년 뒤 150억달러 시장 선다
2016-09-04 23:24:00,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민·관·학계가 함께하는 '...,chosun,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로
2016-09-04 23:20:00,경제,정유업계에 긴장감이 돌고 있다. 올 상반기에는 세계 경기 불황 속에서도 사상 최고 ...,chosun,2016-09-04 23:20:00,중국發 공급과잉·低유가 먹구름… 잘나가던 정유업계 긴장
2016-09-04 22:54:00,경제,지난 3일(현지 시각) 오후 유럽 최대 가전박람회 'IFA 2016'이 열리고 있는...,chosun,2016-09-04 22:54:00,레고 로봇·스마트 캐디·게임 칫솔… 獨서 빛나는 작은 기업들
2016-09-04 22:52:00,nativeAD > LG전자,"""기술력에 대한 자신감이 없다면 우리 가전제품의 핵심 부품을 다른 경쟁 업체에 판매...",chosun,2016-09-04 22:52:00,"""경쟁社에 핵심부품 파는 건, 기술력에 자신감 있기 때문"""


## Preprocessing
0. Datetime (16-09-11 ~ 16-09-17)
1. Remove stopwords (regex, hanja)
2. POS Tagging with KoNLPy, Mecab

In [32]:
import datetime
from konlpy.tag import Mecab
import hanja
import re

In [33]:
mecab = Mecab()
print(mecab.pos(u'네, 안녕하세요 나는 정답'))

[('네', 'IC'), (',', 'SC'), ('안녕', 'NNG'), ('하', 'XSV'), ('세요', 'EP+EF'), ('나', 'NP'), ('는', 'JX'), ('정답', 'NNG')]


In [34]:
def text_cleaning(text):
    text = hanja.translate(text, 'substitution')
    text = re.sub('[^가-힝0-9a-zA-Z\\s]', '', text)
    text = text.replace(u"카드뉴스", '').replace(u"조선일보", '')
    return text

### POS Tagging
일반명사, 고유명사, 외국어 (NNG, NNP, SL)

In [35]:
def tokenize(data):
    return [' '.join(e for e in mecab.nouns(data))]
#     return [' '.join(e for e in mecab.pos(data) if e[1] in ['NNG', 'NNP', 'SL'])]

In [36]:
train['title'] = train['title'].apply(lambda text: text_cleaning(text))
title = [tokenize(each[1]['title']) for each in train.iterrows()]
title

[['전기 저장 년 뒤 억 달러 시장'],
 ['할인 요금제 기간 년'],
 ['중국 발 공급 과잉 저유 먹구름 정유 업계 긴장'],
 ['레고 로봇 스마트 캐디 게임 칫솔 독서 기업'],
 ['경쟁사 핵심 부품 건 기술력 자신 때문'],
 ['올해 내 미 사물 인터넷 서비스 단계 타국 확대'],
 ['영화 예고편 줄'],
 ['영화 예고편 줄'],
 ['원자재 값 급등 중 규제 국내 배터리 사 중고'],
 ['원자재 값 급등 중 규제 국내 배터리 사 중고'],
 ['파업 중 한국 사장 직원 이메일'],
 ['면세점 덩치 한국 신규 곳 적자'],
 ['면세점 덩치 한국 신규 곳 적자'],
 ['보장 보험료 올해 최대'],
 ['교실 밖 아이 선생 질문'],
 ['배달 앱 프랑스 요리 부심'],
 ['오늘 운세 월 일 월요일 음력 월 일 경인'],
 ['미래 첨단 무기 웨어 러블 로봇 주요 기술 확보'],
 ['차세대 무기 체계 레이저 무기 투자 집중'],
 ['국내 최대 규모 지상군 전시회'],
 ['천 년 고도 경주 사극 세트 장'],
 ['소년 세 생일 축하 편지'],
 ['아가 네 세상 게'],
 ['다운 증후군 종업원 은퇴 파티'],
 ['투견 눈물'],
 ['사설 방어 무기 하나 달 소란 정국 마비 나라'],
 ['사설 이번 서경배 억 줄 기업 노블레스 오 블리 주'],
 ['사설 재벌 오너 이름 수십억 연봉'],
 ['만물상 퇴비 냄새'],
 ['박성현 공동 선두'],
 ['터치 코리아 당신 애정 취약 계층'],
 ['트렌드 돋보기 이화'],
 ['가슴 한시'],
 ['이미도 무비 식도'],
 ['조선 주말 하이라이트 모란봉 클럽 외'],
 ['조선 추석 준비 법'],
 ['조선 남자 인생'],
 ['윤성호 허정구 배 아마추어 골프 련패'],
 ['라운드 홀인원 개'],
 ['오늘 경기 년 월 일'],
 ['미 시 풋볼 선수 이번 돼지 양말 논란'],
 ['내일 경기 년 월 일'],
 ['스포츠 브리핑 강정호 재활 경기 홈런'],
 ['정신줄 수 고

In [37]:
train['content'] = train['content'].apply(lambda text: text_cleaning(text))
content = [tokenize(each[1]['content']) for each in train.iterrows()]
content

[['사물 인터넷 연결 사물 인터넷 자동차 냉장고 세탁기 장난감 인터넷 사람 공간 데이터 몸 기술 서비스 에너지 업계 사물 인터넷 개념 시대 에너지 혁명 책 토니 세바 씨 고안 단어 사물 배터리 세상 예언 배터리 휴대 전화 전자 기기 데 자동차 가정 전력 공급 배터리 시작 전문 단어 포장 요즘 에너지 산업 관련 등장 너지 저장 장치 말 배터리 전기 저장 필요 때 발전소 생산 전기 저장 장치 전기 필요 때 공급 전력 사용 효율 시스템 뜻 전기 직류 저장 배터리 교류 직류 변환 전력 변환 장치 시스템 제어 운영 시스템 설명 전자 기기 휴대 전화 배터리 전기 차 동력 제공 전기 안정 공급 정전 시 비상 발전기 역할 초대 배터리 셈 대중 육지 섬 오지 군군 부대 등 태양광 풍력 활용 전력 자급자족 수 전기 시간 태양광 풍력 생산 전력 저장 나중 필요 때 겁니다 가파 에너지 자립 섬 가능 미국 캘리포니아주 일사량 낮 가정 공장 태양광 발전 활용 전기 자체 공급 낮 전력 회사 부하 해 저녁 무렵 수요 급증 전력 공급 차질 시간 전력 공급 보완 조치 의무 낮 생산 전기 중 일부 저장 전력 수요 시간대 공급 겁니다 효율 수급 균형 이유 발전소 필요 일석이조 효과 배터리 시스템 통합 에너지 관리 시스템 건설 전력 공급 발전 등 다양 분야 기술 대표 융합 산업 앞 관련 산업 규모 전망 중요 시장 년 뒤 배 세계 시장 지난해 년 연평균 성장 관련 시장 년 지금 배 억 달러 것 전망 미국 설치 가정 보조금 지급 일본 독일 설치 비용 분 등 발 보급 기차 생산 미국 테슬라 지난해 월 가정 파워 월 출시 눈길 디자인 만 원 비교 가격 출시 주일 만 만 대 예약 판매 등 관심 종류 태양광 발전 생산 전기 전기 요금 쌀 때 공급 전기 저장 때 사용 기기 미국 가구 하루 평균 전력 소비량 용량 만족 만큼 가격 경제 평가 테슬라 년 세계 최대 배터리 공장 기 팩토리 완공 파워 월 대량 생산 가격 가정 산업용 시장 장악 청사진 국내 화학 가정 지난해 차례 뉴 월 독일 뮌헨 인터 솔라 어워드 부

## Training
K-Means Algorithm

In [38]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans, MiniBatchKMeans
from time import time

In [58]:
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,1))
title_flat = [item for sublist in title for item in sublist]
x_list = vectorizer.fit_transform(title_flat)
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,2))
title_flat = [item for sublist in title for item in sublist]
x_list3 = vectorizer.fit_transform(title_flat)
vectorizer = TfidfVectorizer(lowercase=False, ngram_range=(1,3))
title_flat = [item for sublist in title for item in sublist]
x_list4 = vectorizer.fit_transform(title_flat)

In [53]:
from scipy import sparse

In [54]:
x_list = sparse.hstack((x_list, x_list3))
x_list = sparse.hstack((x_list, x_list4))

In [11]:
vectorizer = TfidfVectorizer(lowercase=False)
content_flat = [item for sublist in content for item in sublist]
x_list2 = vectorizer.fit_transform(content_flat)

In [55]:
x_list

<3068x23466 sparse matrix of type '<class 'numpy.float64'>'
	with 45810 stored elements in COOrdinate format>

### Scoring

In [13]:
from sklearn.metrics import silhouette_samples, silhouette_score
from scipy.spatial.distance import cdist, pdist

### Best Silhoutte Score

In [56]:
best_score = 0.0
best_k = 0

In [57]:
for k in range(25, 35):
    km = KMeans(n_clusters=k, n_jobs=-1).fit(x_list)
    score = silhouette_score(x_list, km.labels_)
    if best_score < score:
        best_score = score
        best_k = k
    print("In Clusters =", k, ", Score is : %0.3f" % score)
print("In Clusters =", best_k, ", Best score is : %0.3f" % best_score)

In Clusters = 25 , Score is : 0.014
In Clusters = 26 , Score is : 0.015
In Clusters = 27 , Score is : 0.016


Process ForkPoolWorker-159:
Process ForkPoolWorker-153:
Process ForkPoolWorker-155:
Process ForkPoolWorker-156:
Process ForkPoolWorker-157:
Process ForkPoolWorker-154:
Process ForkPoolWorker-158:
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
Traceback (most recent call last):
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _boo

KeyboardInterrupt: 

### TODO: Using Elbow method

In [16]:
def elbow(df, n):
    kMeansVar = [KMeans(n_clusters=k).fit(df) for k in range(1, n)]
    centroids = [X.cluster_centers_ for X in kMeansVar]
    k_euclid = [cdist(df, cent) for cent in centroids]
    dist = [np.min(ke, axis=1) for ke in k_euclid]
    wcss = [sum(d**2) for d in dist]
    tss = sum(pdist(df)**2)/df.shape[0]
    bss = tss - wcss
    plt.plot(bss)
    plt.show()

### K-Means Algorithm

In [17]:
from matplotlib import pyplot

In [18]:
t0 = time()
km = KMeans(n_clusters=best_k, n_jobs=-1).fit(x_list)
labels = km.labels_
centroids = km.cluster_centers_
print("Done in %0.3fs." % (time() - t0))

Done in 0.720s.


In [19]:
print(km.inertia_)

2854.3164833619053


In [20]:
train['cluster'] = labels
train.head()

,category,content,provider,publishedAt,title,cluster
publishedAt,,,,,,
2016-09-04 23:28:00,경제,모든 사물이 인터넷으로 연결된다는 사물인터넷IoTInternet of Things에...,chosun,2016-09-04 23:28:00,전기 저장했다 꺼내 쓰는 ESS 4년 뒤 150억달러 시장 선다,8
2016-09-04 23:24:00,nativeAD > 기업뉴스,지난달 말 산업통상자원부는 ESS 관련 산업 도약을 위해 민관학계가 함께하는 ESS...,chosun,2016-09-04 23:24:00,ESS 할인 요금제 기간 10년으로,32
2016-09-04 23:20:00,경제,정유업계에 긴장감이 돌고 있다 올 상반기에는 세계 경기 불황 속에서도 사상 최고 영...,chosun,2016-09-04 23:20:00,중국발 공급과잉저유가 먹구름 잘나가던 정유업계 긴장,0
2016-09-04 22:54:00,경제,지난 3일현지 시각 오후 유럽 최대 가전박람회 IFA 2016이 열리고 있는 독일 ...,chosun,2016-09-04 22:54:00,레고 로봇스마트 캐디게임 칫솔 독서 빛나는 작은 기업들,0
2016-09-04 22:52:00,nativeAD > LG전자,기술력에 대한 자신감이 없다면 우리 가전제품의 핵심 부품을 다른 경쟁 업체에 판매하...,chosun,2016-09-04 22:52:00,경쟁사에 핵심부품 파는 건 기술력에 자신감 있기 때문,0


In [21]:
cluster = train.loc[train['cluster']==31]
cluster.title

publishedAt
2016-09-01 03:14:00                        사설 자세 불량이라고 또 처형한 김정은
2016-08-31 03:00:00                   경주 산하기관민간 발굴업체가 천년왕경 플랜 뚝딱
2016-08-31 03:00:00                     프리미엄 버거고급 커피 종합 외식업체로 도약
2016-08-31 03:00:00                    인테리어 내장재 연구 30년 맞춤형 제품 인기
2016-08-30 03:00:00                 식이섬유 풍부한 수제 초코파이 바나나맛 제품도 출시
2016-08-30 03:00:00                     일본 제품보다 훨씬 우수 농진청서 품질 확인
2016-08-30 03:00:00            멥쌀모싯잎동부가 어우러진 자연의 맛 화학첨가물 없는 웰빙식품
2016-08-29 10:40:00                    LG화학 2020년 프리미엄 제품 매출 7조로
2016-09-02 03:00:00                          37억 년 전 지구 최고 화석 발견
2016-09-02 03:00:00                             뉴스룸조숭호북극 자원의 재발견
2016-09-02 03:00:00                              가성비 좋은 제품 한 자리에
2016-09-01 03:00:00                         북방계 부여 동병철검 국내서 첫 발견
2016-09-01 03:00:00                이광표의 근대를 걷는다석굴암의 재발견 그 찬사와 상처
2016-09-01 03:00:00                     프라이팬냄비도 팝니다 전자제품 전문점의 변신
2016-09-01 03:00:00               LoveGift정성 가득 우리 농식품으로 한가위 즐겨요
2016-09-01 03

### Topic Modeling for contents
Latent Dirichlet Allocation (LDA)

In [22]:
from sklearn.decomposition import LatentDirichletAllocation

In [23]:
def print_top_words(model, feature_names, n_top_words):
    for topic_idx, topic in enumerate(model.components_):
        print("Topic #%d:" % topic_idx)
        print(" ".join([feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print()

In [24]:
t0 = time()
lda = LatentDirichletAllocation(n_topics=1, max_iter=5, n_jobs=-1).fit(x_list2[5])
print("Done in %0.3fs." % (time() - t0))

Done in 0.220s.


In [25]:
feature_names = vectorizer.get_feature_names()
print_top_words(lda, feature_names, 5)

Topic #0:
연결 사물 숙환 가전 콜렉터



In [26]:
train['content'][5]

'윤부근 사진  삼성전자 소비자가전CE 부문 대표이사는 3일현지 시각 올해 내에 삼성 스마트TV에 탑재된 사물인터넷IoT 기능을 본격 강화해 진정한 스마트 허브를 구현하는 서비스를 내놓겠다고 말했다 윤 대표는 특히 앞으로 가전 업계는 누가 사물인터넷을 통해 패러다임 전환을 주도해 나가느냐가 사업의 생사를 가를 것이라며 사물인터넷 리더십 확보를 향후 핵심 전략으로 꼽았다\n\n윤 대표는 이날 유럽 최대 가전 전시회 IFA 2016이 열리는 독일 베를린의 삼성전자 전시관에서 기자와 만나 모바일과 TV냉장고 등 가전제품이 자연스럽게 연결되는 방향으로 사물인터넷 제품을 출시하겠다며 지금은 TV와 모바일 기기를 연결하려면 사람이 인위적으로 조작해야 하지만 앞으로는 TV 옆으로 가기만 해도 자동으로 연결돼 스마트폰을 TV 리모컨처럼 사용할 수 있을 것이라고 말했다\n\n삼성전자는 올해 처음으로 IoT 허브를 탑재한 스마트TV를 출시한 데 이어 앞으로 냉장고와 세탁기 등 다른 제품과의 연결성을 강화해 가정의 모든 기기를 연결하는 스마트홈을 본격적으로 구현하겠다는 전략이다 윤 대표는 올해 하반기에 가전 최대 격전지인 미국 시장에서 TV오디오스마트폰을 연결하는 사물인터넷 서비스를 시범 실시한 뒤 단계적으로 연결되는 가전제품과 서비스 국가를 확대할 계획이라고 말했다\n\n윤 대표는 사물인터넷이 23년 후에는 전자 산업의 패러다임을 바꿀 것으로 내다봤다 윤 대표는 현재는 기기를 연결해 제어하는 초기 단계지만 사물인터넷의 꽃은 각종 데이터를 분석한 뒤 소비자에게 다양한 서비스를 제공하는 것이라며 사물인터넷은 물류제조농업교통건강 등 모든 분야에 활용할 수 있기 때문에 앞으로 23년이 지나면 지금의 하드웨어 업체가 과연 살아있을까 하는 생각이 들 정도로 파괴력이 클 것이라고 말했다\n\n한편 윤 대표는 지난달 인수한 미국의 빌트인builtin붙박이 전문 고급 주방가전 업체인 데이코Dacor를 미국뿐만 아니라 유럽에서도 수퍼 프리미엄 브랜드로 내세워 프리미엄 가전과 기업 간 거래B2B 

## Result
1. Model Selection
2. Preprocessing
3. Feature Engineering
4. Parameter Tuning
5. Score Function